# **Notebook 12.1: Self Attention**

This notebook builds a self-attention mechanism from scratch, as discussed in section 12.2 of the book.

Work through the cells below, running each cell in turn. In various places you will see the words "TO DO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.



In [1]:
import numpy as np
import matplotlib.pyplot as plt

The self-attention mechanism maps $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ and returns $N$ outputs $\mathbf{x}'_{n}\in \mathbb{R}^{D}$.  



In [69]:
# Set seed so we get the same random numbers
np.random.seed(3)
# Number of inputs
N = 3
# Number of dimensions of each input
D = 4
# Create an empty list
all_x = []
# Create elements x_n and append to list
for n in range(N):
  all_x.append(np.random.normal(size=(D,1)))
# Print out the list
print(all_x)


[array([[ 1.78862847],
       [ 0.43650985],
       [ 0.09649747],
       [-1.8634927 ]]), array([[-0.2773882 ],
       [-0.35475898],
       [-0.08274148],
       [-0.62700068]]), array([[-0.04381817],
       [-0.47721803],
       [-1.31386475],
       [ 0.88462238]])]


We'll also need the weights and biases for the keys, queries, and values (equations 12.2 and 12.4)

In [70]:
# Set seed so we get the same random numbers
np.random.seed(0)

# Choose random values for the parameters
omega_q = np.random.normal(size=(D,D))
omega_k = np.random.normal(size=(D,D))
omega_v = np.random.normal(size=(D,D))
beta_q = np.random.normal(size=(D,1))
beta_k = np.random.normal(size=(D,1))
beta_v = np.random.normal(size=(D,1))

Now let's compute the queries, keys, and values for each input

In [71]:
# Make three lists to store queries, keys, and values
all_queries = []
all_keys = []
all_values = []
# For every input
for x in all_x:
  # TODO -- compute the keys, queries and values.
  # Replace these three lines
  query= omega_q @ x+ beta_q;
  key= omega_k @ x + beta_k;
  value= omega_v @ x + beta_v;


  all_queries.append(query)
  all_keys.append(key)
  all_values.append(value)

We'll need a softmax function (equation 12.5) -- here, it will take a list of arbitrary numbers and return a list where the elements are non-negative and sum to one


In [73]:
def softmax(items_in):

  # TODO Compute the elements of items_out
  # Replace this line
  e= (np.exp(items_in));
  items_out =e/np.sum(e);
  # print(items_out

  return items_out ;

Now compute the self attention values:

In [84]:
# Create emptymlist for output
all_x_prime = []

# For each output
for n in range(N):
  # Create list for dot products of query N with all keys
  all_km_qn = []
  # Compute the dot products
  for key in all_keys:
    # TODO -- compute the appropriate dot product
    # Replace this line

    dot_product = np.transpose(key) @ all_queries[n];


    # Store dot product
    all_km_qn.append(dot_product)

  # Compute dot product

  attention = softmax(all_km_qn)
  # Print result (should be positive sum to one)
  # print("Attentions for output ", n)
  # print(attention)


  # TODO: Compute a weighted sum of all of the values according to the attention
  # (equation 12.3)
  # Replace this line
  x_prime=np.zeros((D,1))
  # print(all_values.shape)
  for i,value in enumerate(all_values):
    x_prime+=attention[i] * value;


    all_x_prime.append(x_prime)



# Print out true values to check you have it correct
print("x_prime_0_calculated:", all_x_prime[0].transpose())
print("x_prime_0_true: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]")
print("x_prime_1_calculated:", all_x_prime[1].transpose())
print("x_prime_1_true: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]")
print("x_prime_2_calculated:", all_x_prime[2].transpose())
print("x_prime_2_true: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]")
# print(np.array(all_x_prime).transpose())

x_prime_0_calculated: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]
x_prime_0_true: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]
x_prime_1_calculated: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]
x_prime_1_true: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]
x_prime_2_calculated: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]
x_prime_2_true: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]


Now let's compute the same thing, but using matrix calculations.  We'll store the $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ in the columns of a $D\times N$ matrix, using equations 12.6 and 12.7/8.

Note:  The book uses column vectors (for compatibility with the rest of the text), but in the wider literature it is more normal to store the inputs in the rows of a matrix;  in this case, the computation is the same, but all the matrices are transposed and the operations proceed in the reverse order.

In [75]:
# Define softmax operation that works independently on each column
def softmax_cols(data_in):
  # Exponentiate all of the values
  exp_values = np.exp(data_in) ;
  # Sum over columns
  denom = np.sum(exp_values, axis = 0);
  # Replicate denominator to N rows
  denom = np.matmul(np.ones((data_in.shape[0],1)), denom[np.newaxis,:])
  # Compute softmax
  softmax = exp_values / denom
  # return the answer
  return softmax

In [76]:
 # Now let's compute self attention in matrix form
def self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):

  # TODO -- Write this function
  # 1. Compute queries, keys, and values
  queries= omega_q @ X + beta_q;
  keys= omega_k @ X + beta_k;
  values= omega_k @ X + beta_v;
  # 2. Compute dot products
  dot_products= keys.T @ queries;
  # 3. Apply softmax to calculate attentions
  attentions= softmax_cols(dot_products);

  # 4. Weight values by attentions
  X_prime=  values @ attentions ;
  # Replace this line



  return X_prime

In [81]:
# Copy data into matrix
X = np.zeros((D, N))
X[:,0] = np.squeeze(all_x[0])
X[:,1] = np.squeeze(all_x[1])
X[:,2] = np.squeeze(all_x[2])

# Run the self attention mechanism
X_prime = self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

[[ 0.94744244  1.64201168  1.61949281]
 [-0.24348429 -0.08470004 -0.06641533]
 [-0.91310441  4.02764044  3.96863308]
 [-0.44522983  2.18690791  2.15858316]]


If you did this correctly, the values should be the same as above.

TODO:  

Print out the attention matrix
You will see that the values are quite extreme (one is very close to one and the others are very close to zero.  Now we'll fix this problem by using scaled dot-product attention.

In [82]:
# Now let's compute self attention in matrix form
def scaled_dot_product_self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):

  # TODO -- Write this function
  queries= omega_q @ X + beta_q;
  keys= omega_k @ X + beta_k;
  values= omega_k @ X + beta_v;
  # 2. Compute dot products
  dot_products= (keys.T @ queries)/np.sqrt(queries.shape[0]);

  # 3. Apply softmax to calculate attentions
  attentions= softmax_cols(dot_products);

  # 4. Weight values by attentions
  X_prime=  values @ attentions ;
  # 1. Compute queries, keys, and values
  # 2. Compute dot products
  # 3. Scale the dot products as in equation 12.9
  # 4. Apply softmax to calculate attentions
  # 5. Weight values by attentions
  # Replace this line
  # X_prime = np.zeros_like(X);

  return X_prime

In [83]:
# Run the self attention mechanism
X_prime = scaled_dot_product_self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

[[ 0.18259072 -0.97509126 -0.64473677]
 [ 1.05854288 -1.48585222 -1.53880447]
 [-0.61969785 -0.29172843 -0.07996767]
 [-1.5368442  -1.03999165 -0.83297986]]


In [80]:
# It looks like I forgot to import numpy again in the new code block. Let's fix that and re-run the code.

import numpy as np

# Re-define all the functions and rerun the testing part.

# Task 1: Compute Queries, Keys, and Values
def compute_queries_keys_values(all_x, omega_q, omega_k, omega_v, beta_q, beta_k, beta_v):
    all_queries = []
    all_keys = []
    all_values = []

    for x in all_x:
        # Compute query, key, and value using the corresponding parameters
        query = np.dot(omega_q, x) + beta_q
        key = np.dot(omega_k, x) + beta_k
        value = np.dot(omega_v, x) + beta_v

        all_queries.append(query)
        all_keys.append(key)
        all_values.append(value)

    return all_queries, all_keys, all_values

# Task 2: Softmax function
def softmax(items_in):
    exp_items = np.exp(items_in - np.max(items_in))  # subtract max for numerical stability
    items_out = exp_items / np.sum(exp_items)
    return items_out

# Task 3: Compute dot products and attention
def compute_attention_and_weighted_sum(N, all_queries, all_keys, all_values, D):
    all_x_prime = []

    for n in range(N):
        all_km_qn = []
        # Compute the dot products of query n with all keys
        for key in all_keys:
            dot_product = np.dot(all_queries[n].T, key)
            all_km_qn.append(dot_product)

        # Compute attention using softmax
        attention = softmax(all_km_qn)

        # Compute a weighted sum of all values based on attention
        x_prime = np.zeros((D, 1))
        for i, value in enumerate(all_values):
            x_prime += attention[i] * value

        all_x_prime.append(x_prime)

    return all_x_prime

# Task 4: Self-Attention Mechanism
def self_attention(X, omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):
    D, N = X.shape
    all_x = [X[:, n:n+1] for n in range(N)]

    all_queries, all_keys, all_values = compute_queries_keys_values(all_x, omega_q, omega_k, omega_v, beta_q, beta_k, beta_v)

    all_x_prime = compute_attention_and_weighted_sum(N, all_queries, all_keys, all_values, D)

    return np.hstack(all_x_prime)

# Initial parameters setup
np.random.seed(3)
N = 3  # Number of inputs
D = 4  # Dimension of inputs
all_x = [np.random.normal(size=(D, 1)) for _ in range(N)]

# Random parameter initialization
np.random.seed(0)
omega_q = np.random.normal(size=(D, D))
omega_k = np.random.normal(size=(D, D))
omega_v = np.random.normal(size=(D, D))
beta_q = np.random.normal(size=(D, 1))
beta_k = np.random.normal(size=(D, 1))
beta_v = np.random.normal(size=(D, 1))

# Copy data into matrix X
X = np.zeros((D, N))
for i, x in enumerate(all_x):
    X[:, i] = x.squeeze()

# Running the self-attention mechanism
X_prime = self_attention(X, omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

X_prime


array([[ 0.94744244,  1.64201168,  1.61949281],
       [-0.24348429, -0.08470004, -0.06641533],
       [-0.91310441,  4.02764044,  3.96863308],
       [-0.44522983,  2.18690791,  2.15858316]])

TODO -- Investigate whether the self-attention mechanism is covariant with respect to permutation.
If it is, when we permute the columns of the input matrix $\mathbf{X}$, the columns of the output matrix $\mathbf{X}'$ will also be permuted.
